# Multi-Agent Plan Simple Example

We start by installing the library with PIP

In [ ]:
# begin of installation

In [ ]:
!pip install --pre unified-planning[fmap]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.1/537.1 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.7/482.7 KB 18.4 MB/s eta 0:00:00


In [ ]:
# end of installation

## Creating the simple-MA problem

In [ ]:
from unified_planning.shortcuts import *
from unified_planning.model.multi_agent import *
from collections import namedtuple
from unified_planning.io.ma_pddl_writer import MAPDDLWriter


problem = MultiAgentProblem("simple_MA")

#AGENTs
robot_a = Agent("robot_a", problem)
scale_a = Agent("scale_a", problem)

Location = UserType("Location")
door = UserType("door")

home = Object("home", Location)
office = Object("office", Location)
open20 = Object("open20", door)
close20 = Object("close20", door)

#FLUENTs
open = Fluent("open", door=door)
pos = Fluent("pos", loc=Location)

robot_a.add_fluent(pos, default_initial_value=False)
scale_a.add_fluent(open, default_initial_value=False)

#ACTIONs
movegripper = InstantaneousAction("movegripper")
movegripper.add_precondition(pos(office))
movegripper.add_effect(pos(home), True)

open_door = InstantaneousAction("open_door")
open_door.add_precondition(open(close20))
open_door.add_effect(open(open20), True)

robot_a.add_action(movegripper)
scale_a.add_action(open_door)

#OBJECTs
problem.add_object(home)
problem.add_object(office)
problem.add_object(open20)
problem.add_object(close20)

problem.add_agent(robot_a)
problem.add_agent(scale_a)

#INITIAL VALUEs
problem.set_initial_value(Dot(robot_a, pos(office)), True)
problem.set_initial_value(Dot(scale_a, open(close20)), True)

#GOALs
problem.add_goal(Dot(robot_a, pos(home)))
problem.add_goal(Dot(scale_a, open(open20)))

w = MAPDDLWriter(problem)
w.write_ma_domain("simple_ma")
w.write_ma_problem("simple_ma")

## Solving Multi-Agent Planning Problems

The user can enter the following optional parameters in order to configure the search procedure:

`result = planner.solve(problem, "-s", "h")`:

- s N selects the search strategy of FMAP. Currently, FMAP supports only an A search scheme, which is the default value of the tag -s (-s None)

- h N selects the heuristic function(s) used to evaluate the quality of the plans. Currently, the following values for N are supported:

  - 0 - FF heuristic: guides the search through the well-known h_FF heuristic function. This option is available for single-agent planning tasks only.
  - 1 - DTG heuristic: evaluates plans via the heuristic h_DTG.
  - 2 - default option - DTG + Landmarks: this option applies the multi-heuristic search scheme of the MH-FMAP solver by combining the h_DTG and h_Land heuristics to guide the search.
  - 3 - Inc. DTG + Landmarks: incremental multi-heuristic mode that makes use of h_DTG and h_Land.

In [ ]:
with OneshotPlanner(name='fmap') as planner:
    result = planner.solve(problem, None, "1")
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("%s Returned Sequential Plans object: %s" % (planner.name, result.plan.all_sequential_plans()))
        [print(f"{idx} Sequential Plans: {seq_plan}") for idx, seq_plan in enumerate(result.plan.all_sequential_plans())]
        print("Adjacency list:", result.plan.get_adjacency_list)
        print("result:", result)
    else:
        print("Log Error:", result)

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `<ipython-input-5-ef9b04792df7>`, you are using the following planning engine:
  * Engine name: FMAP
  * Developers:  Alejandro Torreño, Oscar Sapena and Eva Onaindia
  * Description: FMAP: A Platform for the Development of Distributed Multi-Agent Planning Systems.

FMAP Returned Sequential Plans object: <generator object PartialOrderPlan.all_sequential_plans at 0x7fdf36e6bf20>
0 Sequential Plans: [scale_a.open_door, robot_a.movegripper]
1 Sequential Plans: [robot_a.movegripper, scale_a.open_door]
Adjacency list: {robot_a.movegripper: [], scale_a.open_door: []}
result: PlanGenerationResult(status=<PlanGenerationResultStatus.SOLVED_SATISFICING: 1>, plan=DiGraph with 2 nodes and 0 edges, engine_name='FMAP', metrics=None, log_messages=[LogMessage(level=<LogLevel.INFO: 2>, message='; Hd